# Kubeflow Pipeline V2 MNIST Example

In [117]:
import kfp
from kfp.v2.dsl import component
from kfp.v2.dsl import pipeline
from kfp.v2.dsl import (
    Input,
    Output,
    Dataset,
    Metrics,
    Model
)
from kfp import compiler

# Components

In [118]:
BASE_IMAGE = 'public.ecr.aws/h3o0w0k1/kubeflow-pipeline-mnist:v3'

In [119]:
@component(base_image=BASE_IMAGE)
def download_datasets(
    train_dataset_output: Output[Dataset],
    test_dataset_output: Output[Dataset]
):
    import torchvision.datasets as dsets
    import os

    os.makedirs(train_dataset_output.path)
    dsets.MNIST(root=train_dataset_output.path, train=True, download=True)

    os.makedirs(test_dataset_output.path)
    dsets.MNIST(root=test_dataset_output.path, train=False, download=True)

In [120]:
@component(base_image=BASE_IMAGE)
def explore_datasets(
    train_dataset_input: Input[Dataset],
    test_dataset_input: Input[Dataset],
    metrics: Output[Metrics]
):
    import torchvision.datasets as dsets

    train = dsets.MNIST(root=train_dataset_input.path, train=True, download=False)
    test = dsets.MNIST(root=test_dataset_input.path, train=False, download=False)

    metrics.log_metric("training samples", len(train))
    metrics.log_metric("test samples", len(test))
    metrics.log_metric("gsd", 5.5)

In [121]:
@component(base_image=BASE_IMAGE)
def train_resnet_model(
    number_of_epochs: int,
    train_batch_size: int,
    learning_rate: float,
    train_dataset_input: Input[Dataset],
    model_output: Output[Model],
    metrics: Output[Metrics]
):
    import torch
    import torch.nn as nn
    import torchvision.datasets as dsets
    from torchvision.transforms import Compose
    from torchvision.transforms import Normalize
    from torchvision.transforms import Resize
    from torchvision.transforms import ToTensor
    from kubeflow_pipeline_sample.resnet.resnet_50 import ResNet50
    from kubeflow_pipeline_sample.training.trainer import train_model

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = ResNet50(in_channels=1, classes=10).to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

    IMAGE_SIZE = 64

    preprocessing = Compose([
        Resize((IMAGE_SIZE, IMAGE_SIZE)), 
        ToTensor(),
        Normalize(mean=(0.5), std=(0.5))
    ])
    train_dataset_clean = dsets.MNIST(root=train_dataset_input.path, train=True, download=False, transform=preprocessing)
    train_loader = torch.utils.data.DataLoader(dataset=train_dataset_clean, batch_size=train_batch_size)

    #losses = train_model(
    #    model=model,
    #    train_loader=train_loader,
    #    criterion=criterion,
    #    optimizer=optimizer,
    #    n_epochs=number_of_epochs,
    #    device=device
    #)
    model_framework: str = f"pytorch:{torch.__version__}"
    model_output.framework = model_framework

    torch.save(model.state_dict(), model_output.path)

In [122]:
@component(base_image=BASE_IMAGE)
def evaluate_resnet_model(
    test_batch_size: int,
    test_dataset_input: Input[Dataset],
    model_input: Input[Model],
    metrics: Output[Metrics]
):
    import torch
    import torchvision.datasets as dsets
    from kubeflow_pipeline_sample.resnet.resnet_50 import ResNet50
    from kubeflow_pipeline_sample.evaluation.evaluate_accuracy import evaluate_accuracy
    from torchvision.transforms import Compose
    from torchvision.transforms import Normalize
    from torchvision.transforms import Resize
    from torchvision.transforms import ToTensor

    IMAGE_SIZE = 64

    preprocessing = Compose([
        Resize((IMAGE_SIZE, IMAGE_SIZE)),
        ToTensor(),
        Normalize(mean=(0.5), std=(0.5))
    ])
    test_dataset_clean = dsets.MNIST(root=test_dataset_input.path, train=False, download=False, transform=preprocessing)
    test_loader = torch.utils.data.DataLoader(dataset=test_dataset_clean, batch_size=test_batch_size)

    model = ResNet50(in_channels=1, classes=10)
    model.load_state_dict(torch.load(model_input.path))
    model.eval()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    accuracy = 0.95#evaluate_accuracy(model, test_loader, test_dataset_clean, device)

    metrics.log_metric("accuracy", accuracy)

# Pipeline

In [123]:
@pipeline(name="end-to-end-pipeline-v2")
def mnist_pipeline(
    number_of_epochs: int = 1,
    train_batch_size: int = 120,
    test_batch_size: int = 120,
    learning_rate: float = 0.1
):
    download_datasets_task = download_datasets()
    explore_datasets_task = explore_datasets(
       train_dataset_input=download_datasets_task.outputs["train_dataset_output"],
       test_dataset_input=download_datasets_task.outputs["test_dataset_output"]
    )
    train_resnet_model_task = train_resnet_model(
       number_of_epochs=number_of_epochs,
       train_batch_size=train_batch_size,
       learning_rate=learning_rate,
       train_dataset_input=download_datasets_task.outputs["train_dataset_output"]
    )
    evaluate_resnet_model_task = evaluate_resnet_model(
       test_batch_size=test_batch_size,
       test_dataset_input=download_datasets_task.outputs["test_dataset_output"],
       model_input=train_resnet_model_task.outputs["model_output"]
    )

# Generate pipeline definition from code

In [124]:
PIPELINE_DEFINITION_FILE_NAME="end_to_end_ml_pipeline_v2.yaml"

In [125]:
compiler.Compiler(mode=kfp.dsl.PipelineExecutionMode.V2_COMPATIBLE).compile(
        pipeline_func=mnist_pipeline,
        package_path=PIPELINE_DEFINITION_FILE_NAME,
        type_check=True
    )

/opt/conda/envs/notebookenv/lib/python3.9/site-packages/kfp/compiler/compiler.py:79: UserWarning: V2_COMPATIBLE execution mode is at Beta quality. Some pipeline features may not work as expected.
  warnings.warn('V2_COMPATIBLE execution mode is at Beta quality.'


# Create Run From Notebook

In [29]:
NAMESPACE="YOUR_NAMESPACE"
INGRESS_GATEWAY="http://istio-ingressgateway.istio-system.svc.cluster.local"
AUTH="YOUR_KUBEFLOW_AUTH_SERVICE_SESSION_BROWSER_COOKIE"
COOKIE="authservice_session="+AUTH

In [30]:
EXPERIMENT_NAME = "End-to-end ML Pipeline experiment"

In [26]:
client = kfp.Client(host=INGRESS_GATEWAY+"/pipeline", cookies=COOKIE)

In [31]:
experiment = client.create_experiment(EXPERIMENT_NAME)
experiment

{'created_at': datetime.datetime(2021, 9, 30, 2, 9, 40, tzinfo=tzlocal()),
 'description': None,
 'id': '9e4520b0-d320-4b11-8962-e26f1b0276e9',
 'name': 'End-to-end ML Pipeline experiment',
 'resource_references': [{'key': {'id': 'mdai-test', 'type': 'NAMESPACE'},
                          'name': None,
                          'relationship': 'OWNER'}],
 'storage_state': 'STORAGESTATE_AVAILABLE'}

In [32]:
run_name = f"e2e-pipeline-run-from-notebook-{datetime.now()}"
run = client.run_pipeline(experiment.id, run_name, PIPELINE_DEFINITION_FILE_NAME)